## MEDLINE Papers Network Analysis

Every year, MELINE(Medical Literature Analysis and Retrieval System Online) would publish a sample of acamdemic papers covering the life sciences and medcine. 

You can find the MEDLINE dataset here: [ftp://ftp.nlm.nih.gov/nlmdata/sample/medline/](ftp://ftp.nlm.nih.gov/nlmdata/sample/medline/). There are eight separate XML files with the total size of 928m. 

In this notebook, I'm going to investigate three topics about the network analysis:

[Part1: Community Detection](#Part1: Community Detection)

Considering the languages used, which group of users is more likely to publish papers together? Is this true that the smaller the language group is, the higher the clustering effect would occur?

[Part2: Co-occurence of Attributes](#Part2: Co-occurence of Attributes)



Considering the descriptions and keywords of the papers, what combinations of descriptions or keywords are more likely to appear together?

[Part3: Link Prediction](#Part3: Link Prediction)

Although this is not a classic social network, can we build a model to predict waht combinations of authors are more likely to publish a paper together(having an edge)?

Dependencies: 

networkX. Pure python implementation of graph algorithms, works fine with this dataset, but if you are analyzing bigger graphs, the scalability could be a problem. iGraph is not better but GraphX would be a good alternative, coming with Pregel API is even better.

H2O. Very fast data processing and model training thanks to its structure of compressing the data and store it on the JVM heap. The documentaion is complex but it is promising and offers H2O with hadoop/Spark to scale.

### Preperation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
% matplotlib inline

# name your own file path
dirpath = "/Users/apple/Downloads/datasets/aas/medline/"

In [2]:
import xml.etree.cElementTree as ET
import codecs

# use iterparse to extract info from xml files
def extract_papers(fs):
    '''
    parameter: xml files
    return: dictionary of papers
            key: PMID
            value: list of authors
                   list of language used
                   list of descriptions
                   list of keywords
    '''
    papers = {}
    for f in fs:
        with codecs.open(f, 'r') as fi:
            path = []
            for event, elem in ET.iterparse(fi, events=("start", "end")):
                if event == "start":
                    path.append(elem.tag)
                    if elem.tag == 'PMID':
                        k = elem.text
                        papers[k] = {'auth':[], 'lang':[], 'description':[], 'keyword':[]}
                    elif 'Author' in path:
                        if elem.tag == 'LastName':
                            papers[k]['auth'].append([elem.text])
                        if elem.tag == 'ForeName' and elem.text is not None:
                            papers[k]['auth'][-1].append(elem.text)
                    elif elem.tag == 'Language' and elem.text is not None:
                        langs = elem.text
                        if len(langs)%3 != 0:
                            print '{0}: incorrect/no languages -> {1}'.format(k, langs)
                        else:
                            for i in range(len(langs)/3):
                                papers[k]['lang'].append(langs[3*i:(3*i+3)])
                    elif elem.tag == 'DescriptorName':
                        papers[k]['description'].append(elem.text)
                    elif elem.tag == 'Keyword':
                        papers[k]['keyword'].append(elem.text)
                elif event == "end":
                    path.pop()
                elem.clear()
    for k,v in papers.iteritems():
        if len(v['auth']) > 0:
            papers[k]['auth'] = [' '.join(auth) for auth in papers[k]['auth'] if auth[0] is not None]
    return {k:v for k,v in papers.iteritems() if len(papers[k]['auth']) > 0}


files = []
for i in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
    fn = dirpath+'medsamp2016'+i+'.xml'
    files.append(fn)
    
papers = extract_papers(files)

In [3]:
# take a look at the result
# filter the result with more than one language used
[(k,v) for k,v in papers.iteritems() if len(v['lang']) > 1][:2]

[('23515729',
  {'auth': ["Mont'Alverne",
    'Galdino Lara Maia',
    'Pinheiro Marcela Cunha',
    u'Levy C\xedntia Souto',
    'Vasconcelos Glauber Gean de',
    u'Souza Neto Jo\xe3o David de',
    u'Mej\xeda Juan Alberto Cosquillo'],
   'description': ['Cardiomyopathy, Dilated',
    'Cross-Sectional Studies',
    'Exercise Test',
    'Female',
    'Heart Transplantation',
    'Humans',
    'Intraoperative Complications',
    'Male',
    'Middle Aged',
    'Postoperative Complications',
    'Reference Values',
    'Retrospective Studies',
    'Tachycardia',
    'Time Factors',
    'Treatment Outcome',
    'Ventricular Dysfunction, Right'],
   'keyword': [],
   'lang': ['eng', 'por']}),
 ('14360229',
  {'auth': ['HOHORST H J'],
   'description': ['Antibodies',
    'Antigens',
    'Bacteria',
    'Blood Group Antigens',
    'Escherichia',
    'Humans',
    'O Antigens',
    'Salmonella'],
   'keyword': ['ANTIGENS AND ANTIBODIES',
    'BLOOD GROUPS',
    'ESCHERICHIA',
    'SALMONELLA'

### Part1: Community Detection

In [4]:
from itertools import combinations
import networkx as nx

def create_graph():
    '''
    return:
    a graph with author as node
    coauthorship as edge
    language used as attribute
    '''
    G = nx.Graph()
    for v in papers.itervalues():
        langs = v['lang']
        auths = v['auth']
        if len(auths) > 1:
            G.add_nodes_from(auths)
            for comb in combinations(auths, 2):
                for lang in langs:
                    G.add_edge(comb[0], comb[1], lang=lang)
        else:
            G.add_node(auths[0])
    print 'Number of nodes: {}'.format(G.number_of_nodes())
    print 'Number of edges: {}'.format(G.number_of_edges())
    return G

G_auth = create_graph()

Number of nodes: 476558
Number of edges: 1090012


In [5]:
from pprint import pprint

langList = {}
for v in papers.itervalues():
    for lang in v['lang']:
        if langList.get(lang) is None:
            langList[lang] = set([])
        for auth in v['auth']:
                langList[lang].add(auth)

# count the lanuage used in the papers
pprint(sorted([(k,len(v)) for (k,v) in langList.iteritems()], key=lambda x: -x[1])[:20])

[('eng', 397167),
 ('fre', 16274),
 ('ger', 14503),
 ('spa', 9819),
 ('rus', 9669),
 ('ita', 7879),
 ('pol', 6307),
 ('chi', 6161),
 ('jpn', 4757),
 ('por', 3675),
 ('cze', 2441),
 ('hun', 1546),
 ('rum', 1321),
 ('dut', 1199),
 ('bul', 842),
 ('und', 818),
 ('swe', 791),
 ('hrv', 690),
 ('ukr', 515),
 ('dan', 493)]


In [6]:
'''
Count the languages with the highest cluster coefficient. Since the group with less than 1000 entities
may have extreme stats, I would leave out the languages used less than 1000 times
'''
langs = {k:v for k,v in langList.iteritems() if len(v) > 1000}
cc = {k:nx.average_clustering(G_auth,v) for k,v in langs.iteritems()}
sorted(cc.iteritems(), key=lambda x: -x[1])[:10]

[('chi', 0.7984509515953556),
 ('rum', 0.7478133083091381),
 ('eng', 0.7253124238106683),
 ('por', 0.6679958821863696),
 ('spa', 0.6059905017850732),
 ('rus', 0.5849271128734129),
 ('pol', 0.5642259921085655),
 ('jpn', 0.5472025628319449),
 ('fre', 0.5434637970409696),
 ('hun', 0.5153554056161257)]

According to the numbers, there is no significant evidence of correlation between the size of the user group and the clustering coefficient. Considering the differences between Chinese/Rumanian and English, the culture of people using different language and the language distances, the explanation could be complex. 

### Part2: Co-occurence of Attributes

In [7]:
from collections import Counter

# count the ocurrances of descriptions and keywords
counter_descriptor = Counter()
counter_keyword = Counter()
for v in papers.itervalues():
    counter_descriptor.update(v['description'])
    counter_keyword.update(v['keyword'])

In [8]:
# take a look at the top descriptions
counter_descriptor.most_common(10)

[('Humans', 102582),
 ('Female', 37088),
 ('Male', 35074),
 ('Animals', 32425),
 ('Adult', 22722),
 ('Middle Aged', 18004),
 ('Aged', 12365),
 ('Adolescent', 10009),
 ('Child', 9782),
 ('Rats', 8858)]

In [9]:
# take a look at the top keywords
counter_keyword.most_common(10)

[('Population', 1826),
 ('Demographic Factors', 1745),
 ('Developing Countries', 1659),
 ('Population Dynamics', 1376),
 ('Economic Factors', 1205),
 ('Developed Countries', 1122),
 ('Research Methodology', 1026),
 ('Population Characteristics', 932),
 ('Asia', 780),
 ('EXPERIMENTAL LAB STUDY', 719)]

In [10]:
G_des = nx.Graph()
for v in papers.itervalues():
    for comb in combinations(v['description'], 2):
        G_des.add_node(comb[0])
        G_des.add_node(comb[1])
        G_des.add_edge(comb[0], comb[1])

In [11]:
from datetime import datetime

# calculate the average length of the shortest paths
# if you need more detailed timing and performance info, use cProfile module
t0 = datetime.now()

def average_shortest_path_length(G):
    length = 0
    for u,v in G.edges_iter():
        if u is not None and v is not None:
            spl = nx.shortest_path_length(G, u, v)
            length += spl
    n = len(G.edges())
    return length / float(n)

avg_length = average_shortest_path_length(G_des)
    
t1 = datetime.now()
print t1-t0

0:08:00.965552


In [12]:
avg_length

0.9968995591316296

In [13]:
# count the occurance of pairs of descriptions
counter_edge = Counter()
for v in papers.itervalues():
    # sort the list of description to make sure that there would not be tuples with inverse order
    sorted_des = sorted(v['description'])
    for e in combinations(sorted_des, 2):
        if e:
            counter_edge.update((e,))
counter_edge.most_common(10) 

[(('Female', 'Humans'), 32219),
 (('Humans', 'Male'), 28590),
 (('Adult', 'Humans'), 22653),
 (('Female', 'Male'), 21715),
 (('Humans', 'Middle Aged'), 17968),
 (('Adult', 'Female'), 15806),
 (('Adult', 'Male'), 15163),
 (('Male', 'Middle Aged'), 13452),
 (('Female', 'Middle Aged'), 13247),
 (('Aged', 'Humans'), 12310)]

In [14]:
from scipy.stats import chi2_contingency

# for each pair of description, run the chi square independence test

def calculate_chi2(T, c1, c2):
    table = {}
    for u,v in c2:
        crosstab = np.array([[0,0],[0,0]])
        val = c2[(u,v)]
        crosstab[1][1] += val
        crosstab[0][1] += (c1[u]-val)
        crosstab[1][0] += (c1[v]-val)
        crosstab[0][0] = T - crosstab[0][1] - crosstab[1][1] - crosstab[1][0]
        chisq = chi2_contingency(crosstab)[0]
        table[(u,v)] = chisq
    return table

T = len(papers)
chi2_table = calculate_chi2(T, counter_descriptor, counter_edge)

In [15]:
# take a look at the distribution of the chi2_table
print 'total number of pairs of desriptions: {}'.format(len(chi2_table))
pprint(np.histogram(chi2_table.values()))

p = 80
print '{0}% of the values fall into [0, {1}]'.format(p, np.percentile(chi2_table.values(), p))

total number of pairs of desriptions: 2429977
(array([2427178,    1812,     599,     187,      81,      46,      38,
            22,       5,       9]),
 array([  9.65653862e-11,   2.02875925e+04,   4.05751851e+04,
         6.08627776e+04,   8.11503701e+04,   1.01437963e+05,
         1.21725555e+05,   1.42013148e+05,   1.62300740e+05,
         1.82588333e+05,   2.02875925e+05]))
80% of the values fall into [0, 66.6193527632]


In [16]:
# take 67 as the threshold, select the pairs of descriptions with ch2 value greater than 67
selected_pairs = [k for k,v in chi2_table.iteritems() if v > 67]

# construct the selected graph for analysis
G_sel = nx.Graph()
for u,v in selected_pairs:
    G_sel.add_nodes_from([u,v])
    G_sel.add_edges_from([(u,v)])
print 'Number of nodes: {}'.format(len(G_sel.nodes()))
print 'Number of edges: {}'.format(len(G_sel.edges()))

Number of nodes: 22044
Number of edges: 484434


In [17]:
avg_length_sel = average_shortest_path_length(G_sel)
avg_length_sel

0.9999525219121697

The average shortest path length does not change a lot after the pairs of descriptions with low chi square values are removed - the connections between each pair of nodes(the authors) are already strong enough, which is not exactly what I have expected. On the other hand, having a stable stats after some of the nodes and edges are removed is a sign of a stable network, indicating some attributes of a small-world network.

### Part3: Link Prediction

In [18]:
def train_vali_split(G):
    '''
    parameter: the co-authorship graph
    return: training graph and validation graph
    split ratio: 7:3
    '''
    G_vali = nx.Graph()
    vali_nodes = [n for n in G.nodes_iter() if np.random.random() <= 0.3]
    G_vali = G.subgraph(vali_nodes)
    G.remove_nodes_from(vali_nodes)
    print 'Number of nodes in training graph: {}'.format(len(G.nodes()))
    print 'Number of edges in training graph: {}'.format(len(G.edges()))    
    print 'Number of nodes in validation graph: {}'.format(len(G_vali.nodes()))
    print 'Number of edges in validation graph: {}'.format(len(G_vali.edges()))
    return G, G_vali

G_train, G_vali = train_vali_split(G_auth)

Number of nodes in training graph: 333958
Number of edges in training graph: 535985
Number of nodes in validation graph: 142600
Number of edges in validation graph: 97827


#### Change of Split Method

It seems that although the nodes ratio is close to 7:3, it is not a good idea to use subgraph() to do the split, since the random selection of nodes would lead to a much less coupled graph wiht much fewer edges. This situation would lead to the underfit of the validation set, since it is less likely to have edges in the validation graph. 

Thankfully we have the original papers data, instead I'm going to construct the train/validation graph from scratch.

In [19]:
def split_from_origin(papers):
    '''
    parameter: dictionary of paper: authors
    return: training graph and validation graph
    using number of co-author(s) as weight
    split ratio: 7:3
    '''
    G_train = nx.Graph()
    G_vali = nx.Graph()
    for k,v in papers.iteritems():
        auths = v['auth']
        if np.random.random() <= 0.7:
            G_train.add_nodes_from(auths)
            for comb in combinations(auths, 2):
                G_train.add_edge(comb[0], comb[1], weight=len(auths))
        else:
            G_vali.add_nodes_from(auths)
            for comb in combinations(auths, 2):
                G_vali.add_edge(comb[0], comb[1], weight=len(auths))
    print 'Number of nodes in training graph: {}'.format(len(G_train.nodes()))
    print 'Number of edges in training graph: {}'.format(len(G_train.edges()))    
    print 'Number of nodes in validation graph: {}'.format(len(G_vali.nodes()))
    print 'Number of edges in validation graph: {}'.format(len(G_vali.edges()))
    return G_train, G_vali

G_train, G_vali = split_from_origin(papers)

Number of nodes in training graph: 354681
Number of edges in training graph: 773047
Number of nodes in validation graph: 169446
Number of edges in validation graph: 337784


In [46]:
# now the train/validation graphs look much better, let's get started!
import math

def get_data(G):
    
    N = len(G.nodes())
    print 'Number of nodes: {}'.format(N)
    
    data = {}
    data['label'] = {}
    flist = ['Jaccard', 'Cosine', 'Adar', 'common_neighbors', 'PageRank', 'kNN1', 'kNN2']
    for f in flist:
        data[f] = {}
    
    # personalized pagerank, setting the damping parameter as 0.8
    # return a dictionary of node: score
    pagerank = nx.pagerank(G, alpha=0.8, weight='weight')
    
    # since the possible edge set would be enourmous, we need to use previous pagerank
    # to filter the nodes with highest score
    N_high = N/200
    N_candidate = N_high * (N_high-1) / 2
    print 'Number of edges to calculate: {}'.format(N_candidate)
    
    nodes = dict(sorted(pagerank.iteritems(), key=lambda x: -x[1])[:N_high])
    k = 0
    K = 0
    for u,v in combinations(nodes.keys(), 2):
        
        un = [_ for _ in nx.all_neighbors(G, u)]
        vn = [_ for _ in nx.all_neighbors(G, v)]
        cn = set(un) & set(vn)
        union = set(un) | set(vn)
        un_len = len(un)
        vn_len = len(vn)
        cn_len = len(cn)
        
        data['common_neighbors'][(u,v)] = cn_len
        data['Jaccard'][(u,v)] = float(cn_len) / len(union)
        data['Cosine'][(u,v)] = float(cn_len) / (un_len * vn_len)
        
        if cn_len == 1 or cn_len == 0:
            data['Adar'][(u,v)] = 0
        else:
            data['Adar'][(u,v)] = 1.0 / math.log(cn_len)
        
        # the ways to calculate kNN weights are various, per se wa+wb, wa * wb
        # even deeper (2nd level or more) connections can be taken into account
        # for the sake of performance, I chose the 1st level calculation as weights
        data['kNN1'][(u,v)] = (1 / math.sqrt(1 + un_len)) + (1 / math.sqrt(1 + vn_len))
        data['kNN2'][(u,v)] = (1 / math.sqrt(1 + un_len)) * (1 / math.sqrt(1 + vn_len))
        data['PageRank'][(u,v)] = nodes[u] * nodes[v]
        
        if G.has_edge(u,v) is True:
            data['label'][(u,v)] = 1
        else:
            data['label'][(u,v)] = 0
        
        k += 1
        if k % (N_candidate/10) == 0:
            K += 10
            print 'Current progress: {}%'.format(K) 
    return data

In [47]:
data_train = get_data(G_train)

Number of nodes: 354681
Number of edges to calculate: 1570878
Current progress: 10%
Current progress: 20%
Current progress: 30%
Current progress: 40%
Current progress: 50%
Current progress: 60%
Current progress: 70%
Current progress: 80%
Current progress: 90%
Current progress: 100%


In [49]:
# just to ensure the index is correct, which is crucial to the problem
df_train = pd.DataFrame(data_train)
df_train.head()

,Adar,Cosine,Jaccard,PageRank,common_neighbors,kNN1,kNN2,label
"(ARNOULD P, Abayomi A)",0,0,0,9.442268e-11,0,0.634845,0.100504,0
"(ARNOULD P, Abe H)",0,0,0,1.279890e-10,0,0.462433,0.043033,0
"(ARNOULD P, Abe K)",0,0,0,1.612973e-10,0,0.457368,0.041345,0
"(ARNOULD P, Abe T)",0,0,0,1.474687e-10,0,0.460333,0.042333,0
"(ARNOULD P, Abolmaesumi Purang)",0,0,0,1.243724e-10,0,0.591532,0.086066,0


In [50]:
data_vali = get_data(G_vali)

Number of nodes: 169446
Number of edges to calculate: 358281
Current progress: 10%
Current progress: 20%
Current progress: 30%
Current progress: 40%
Current progress: 50%
Current progress: 60%
Current progress: 70%
Current progress: 80%
Current progress: 90%
Current progress: 100%


In [51]:
df_vali = pd.DataFrame(data_vali)
df_vali.head()

,Adar,Cosine,Jaccard,PageRank,common_neighbors,kNN1,kNN2,label
"(Abe T, Akbulut H)",0,0,0,5.528048e-10,0,0.387298,0.033333,0
"(Abe T, BONNAL J)",0,0,0,5.470430e-10,0,0.430611,0.038925,0
"(Abe T, Berg T)",0,0,0,5.673243e-10,0,0.406450,0.035806,0
"(Abe T, Bird E D)",0,0,0,5.742763e-10,0,0.347317,0.028172,0
"(Abe T, Bloom S R)",0,0,0,9.635237e-10,0,0.321550,0.024845,0


In [57]:
df_train.to_csv('train.csv', index=False)
df_vali.to_csv('validate.csv', index=False)

In [1]:
import h2o
h2o.init()

H2O cluster uptime:,7 minutes 53 seconds 98 milliseconds
H2O cluster version:,3.8.1.3
H2O cluster name:,light
H2O cluster total nodes:,1
H2O cluster total free memory:,1.56 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [2]:
import os

basedir = os.getcwd()
fr_train = h2o.import_file(path=basedir+"/train.csv")
fr_train.head(5)


Parse Progress: [##################################################] 100%


Adar,Cosine,Jaccard,PageRank,common_neighbors,kNN1,kNN2,label
0,0,0,9.44227e-11,0,0.634845,0.100504,0
0,0,0,1.27989e-10,0,0.462433,0.0430331,0
0,0,0,1.61297e-10,0,0.457368,0.0413449,0
0,0,0,1.47469e-10,0,0.460333,0.0423334,0
0,0,0,1.24372e-10,0,0.591532,0.0860663,0


In [3]:
fr_vali = h2o.import_file(path=basedir+"/validate.csv")
fr_vali.head(5)


Parse Progress: [##################################################] 100%


Adar,Cosine,Jaccard,PageRank,common_neighbors,kNN1,kNN2,label
0,0,0,5.52805e-10,0,0.387298,0.0333333,0
0,0,0,5.47043e-10,0,0.430611,0.0389249,0
0,0,0,5.67324e-10,0,0.40645,0.0358057,0
0,0,0,5.74276e-10,0,0.347317,0.0281718,0
0,0,0,9.63524e-10,0,0.32155,0.0248452,0


In [5]:
fr_train['label'] = fr_train['label'].asfactor()
fr_vali['label'] = fr_vali['label'].asfactor()

from h2o.estimators.gbm import H2OGradientBoostingEstimator


gbm = H2OGradientBoostingEstimator(distribution='bernoulli',
                                   ntrees=100,
                                   max_depth=5,
                                   balance_classes=True,
                                   learn_rate=0.01)

gbm.train(x=fr_train.columns[:-1],
          y='label', 
          training_frame=fr_train,
          validation_frame=fr_vali)

gbm.show()


gbm Model Build Progress: [##################################################] 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1458712020197_2

Model Summary: 


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,40353.0,5.0,5.0,5.0,28.0,32.0,29.36




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.491430863128
R^2: -0.965723452513
LogLoss: 2.38680414851
AUC: 0.998721217075
Gini: 0.997442434149

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.00238006530015: 


,0,1,Error,Rate
0,1551478.0,16225.0,0.0103,(16225.0/1567703.0)
1,9875.0,1557829.0,0.0063,(9875.0/1567704.0)
Total,1561353.0,1574054.0,0.0083,(26100.0/3135407.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100284,0.0055700,97.9009226,97.9009226,0.4052324,0.4052324,0.9817937,0.9817937,9690.0922581,9690.0922581
,2,0.0201015,0.0045264,0.8032986,49.2442543,0.0033250,0.2038323,0.0080917,0.9898854,-19.6701436,4824.4254284
,3,0.0459751,0.0010618,0.2345554,21.6629096,0.0009709,0.0896673,0.0060688,0.9959541,-76.5444618,2066.2909649
,4,0.0514959,0.0010598,0.0,19.3404579,0.0,0.0800542,0.0,0.9959541,-100.0,1834.0457872
,5,0.1468512,0.0010533,0.0,6.7820627,0.0,0.0280724,0.0,0.9959541,-100.0,578.2062716
,6,0.1790131,0.0009511,0.0,5.5635818,0.0,0.0230288,0.0,0.9959541,-100.0,456.3581829
,7,0.2008675,0.0004827,0.0617093,4.9649788,0.0002554,0.0205511,0.0013486,0.9973028,-93.8290666,396.4978835
,8,0.5254256,0.0004669,0.0041552,1.9006524,0.0000172,0.0078672,0.0013486,0.9986514,-99.5844757,90.0652409
,9,0.8418002,0.0004644,0.0021314,1.1871294,0.0000088,0.0049138,0.0006743,0.9993257,-99.7868638,18.7129379
,10,0.9138972,0.0004638,0.0093528,1.0942150,0.0000387,0.0045292,0.0006743,1.0,-99.0647205,9.4214982




Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error,validation_MSE,validation_logloss,validation_AUC,validation_lift,validation_classification_error
,2016-03-22 22:56:21,0.058 sec,0.0,0.4979831,3.1030538,0.5,1.0,0.4999998,0.0041266,0.0276948,0.5,1.0,0.9958608
,2016-03-22 22:56:27,6.351 sec,1.0,0.4979437,3.0933879,0.9974892,72.0553396,0.0106088,0.0041261,0.0275812,0.9950889,72.7052913,0.0008625
,2016-03-22 22:56:41,19.677 sec,9.0,0.4976146,3.0193144,0.9975125,93.9624102,0.0104136,0.0041225,0.0267308,0.9962087,70.8427796,0.0008625
,2016-03-22 22:57:14,52.389 sec,28.0,0.4967236,2.8615534,0.9975979,95.8447957,0.0099081,0.0041139,0.0250174,0.9973811,65.5954657,0.0008485
,2016-03-22 22:58:31,2 min 10.060 sec,75.0,0.4936848,2.5371535,0.9986898,98.4878459,0.0085265,0.0040872,0.0217288,0.9978231,97.8192477,0.0008094
,2016-03-22 22:59:33,3 min 11.958 sec,100.0,0.4914309,2.3868041,0.9987212,97.8302515,0.0083243,0.0040680,0.0203167,0.9979633,97.9009226,0.0008094



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Jaccard,32645676.0000000,1.0,0.9871298
Adar,241428.3125000,0.0073954,0.0073002
PageRank,70037.6875000,0.0021454,0.0021178
kNN1,57721.3164062,0.0017681,0.0017454
Cosine,32514.1152344,0.0009960,0.0009832
kNN2,23931.5761719,0.0007331,0.0007236
common_neighbors,0.0,0.0,0.0


#### Link prediction result:

Without feature engineering and grid search, the model is able to provide prediction with accuracy of 99.92% and AUC of 99.79% which proves that the choices of the features are correct! Even facing the sparsity of social network, this model ahieve its goal with basic attributes calculations and even 'out-of-date' PageRank algorithm. On this dataset, I would say the link prediction problem is solved.